In [1]:
from pre import imputation


import pandas as pd
import numpy as np

In [2]:
df_x_train = pd.read_csv('Data/X_train.csv')
df_y_train = pd.read_csv('Data/y_train.csv')
df_test = pd.read_csv('Data/X_test.csv')
df_train_id = df_x_train['id']
df_test_id = df_test['id'].copy()

df_train = pd.merge(df_x_train, df_y_train, on='id')

df_train = df_train.drop("id", axis=1) #we don't need id column for training set, only for final testing


# Preprocess

In [5]:
# Imputation of values

estimator_params = {
    'n_estimators': 100,
    'max_depth': 3,
    'learning_rate': 0.1
    # Add other XGBoost parameters here
}

mice_params = {
    'max_iter': 10,
    'imputation_order': 'descending'
    # Add other MICE parameters here
}

# Call the function with the additional parameters
df_train_imputed = imputation.impute_data_with_mice_V2(
    df_train.drop(columns=['y']),
    estimator_params=estimator_params,
    mice_params=mice_params
)
df_train_imputed = pd.concat([df_train_imputed, df_train['y']], axis=1)

df_test_imputed = imputation.impute_data_with_mice_V2(
    df_test.drop(columns=["id"]),
    estimator_params=estimator_params,
    mice_params=mice_params
)

df_test_imputed = pd.concat([df_test_imputed, df_test['id']], axis=1)

In [4]:
# Outlier removal
from pre import outliers_detection
columns_to_exclude = ['y']

feature_columns = [col for col in df_train_imputed.columns if col not in columns_to_exclude]
input_mat = df_train_imputed[feature_columns].values

processed_data = outliers_detection.outliers_z_score(input_mat)

df_train_outlier = pd.DataFrame(processed_data, columns=feature_columns)

for col in columns_to_exclude:
    df_train_outlier[col] = df_train_imputed[col]



input_mat_test = df_test_imputed.drop(columns=['id']).values
processed_data_test = outliers_detection.outliers_z_score(input_mat_test)

df_test_outlier = pd.DataFrame(processed_data_test, columns=feature_columns)

df_test_outlier = pd.concat([df_test_outlier, df_test_imputed['id']], axis=1)


NameError: name 'df_train_imputed' is not defined

In [ ]:
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest

X = df_train_outlier.drop(columns=['y'])  # Extract features
y = df_train_outlier['y']  # Extract the target

k = 5  # Change this to the desired number of features

selector = SelectKBest(score_func=f_classif, k=k)

X_new = selector.fit_transform(X, y)

selected_indices = selector.get_support(indices=True)

selected_features_df = pd.DataFrame(X_new, columns=X.columns[selected_indices])
selected_features_df_test = df_test_imputed.iloc[:, selected_indices]


In [ ]:
df_processed = pd.concat([selected_features_df, df_train_outlier['y']], axis=1)
df_test_processed = pd.concat([selected_features_df_test, df_test_imputed['id']], axis=1)


In [ ]:
#If no feature selection

df_processed = df_train_imputed
df_test_processed = df_test_imputed

# Modeling

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_processed.drop(columns=['y']), df_processed['y'], test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

# Construct pipeline & fit model
pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('lasso', Lasso(alpha=0.1, fit_intercept=True))
])
pipe.fit(X_train, y_train)

# Print coefficients
lasso_coef = pd.Series(pipe.named_steps['lasso'].coef_)
lasso_coef.index = pd.Index(X_train.columns)
print(lasso_coef.sort_values(ascending=False, key=abs))

# Important features
important_features = lasso_coef[lasso_coef != 0].index
print('Number of important features: ', important_features.shape[0])

# Calculate R2 score on test split
pred = pipe.predict(X_test)
r2 = r2_score(y_test, pred)
print(f'R^2 Score on Test Set: {r2:.2f}')


x458    2.007553
x141    1.841784
x89     1.449619
x194   -1.407826
x350    1.302523
          ...   
x111    0.000000
x112   -0.000000
x116   -0.000000
x120    0.000000
x252   -0.000000
Length: 832, dtype: float64
Number of important features:  347
R^2 Score on Test Set: 0.34


In [ ]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

# Define features and target
X = df_processed.drop(columns=['y'])
y = df_processed['y']

# Construct the pipeline & Lasso model
pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('lasso', Lasso(alpha=0.1, fit_intercept=True))
])

# Perform cross-validation (e.g., 5-fold cross-validation)
cv_scores = cross_val_score(pipe, X, y, cv=5, scoring='r2')
best_model = pipe.fit(X, y)  # Train the best model on the entire training data
print('Cross-Validation R^2 Scores:', cv_scores)

# Calculate the mean and standard deviation of the R^2 scores
mean_r2 = cv_scores.mean()
std_r2 = cv_scores.std()

print(f'Mean R^2 Score: {mean_r2:.2f}')
print(f'Standard Deviation of R^2 Scores: {std_r2:.2f}')



X_test = df_test_processed.drop('id', axis=1).astype(float)  # Ensure data type is float
predictions = best_model.predict(X_test)

submission = pd.DataFrame({'id': df_test_processed['id'], 'y': predictions})
submission.to_csv('submissionLinReg.csv', index=False)


c:\Users\alois\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.334e+01, tolerance: 9.053e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\alois\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.730e+01, tolerance: 9.215e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\alois\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.887e+01, toleranc

Cross-Validation R^2 Scores: [0.30723969 0.24803328 0.36113586 0.29245828 0.22079943]
Mean R^2 Score: 0.29
Standard Deviation of R^2 Scores: 0.05


In [ ]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV, cross_val_score
import pandas as pd

# Define features and target
X = df_processed.drop(columns=['y'])
y = df_processed['y']

# Create a pipeline with a Lasso model and a scaler
pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('lasso', Lasso())
])

# Define a grid of alpha values to search
param_grid = {
    'lasso__alpha': [0.001, 0.01, 0.1, 1.0, 10.0]
}

# Create a GridSearchCV object
grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='r2', n_jobs=-1)

# Fit the GridSearchCV to the data to find the best hyperparameters
grid_search.fit(X, y)

# Get the best model with the tuned hyperparameters
best_model = grid_search.best_estimator_

# Perform cross-validation with the best model
cv_scores = cross_val_score(best_model, X, y, cv=5, scoring='r2')
print('Cross-Validation R^2 Scores:', cv_scores)

# Calculate the mean and standard deviation of the R^2 scores
mean_r2 = cv_scores.mean()
std_r2 = cv_scores.std()
print(f'Mean R^2 Score: {mean_r2:.2f}')
print(f'Standard Deviation of R^2 Scores: {std_r2:.2f}')

# Use the best model to make predictions on the test data
X_test = df_test_processed.drop('id', axis=1).astype(float)  # Ensure data type is float
predictions = best_model.predict(X_test)

# Create a submission DataFrame
submission = pd.DataFrame({'id': df_test_processed['id'], 'y': predictions})
submission.to_csv('submissionLinReg.csv', index=False)


Cross-Validation R^2 Scores: [0.35661827 0.31213773 0.40096122 0.35778794 0.22811881]
Mean R^2 Score: 0.33
Standard Deviation of R^2 Scores: 0.06


# Submission Real test set

In [ ]:

predictions = model.predict(df_test_processed.drop('id', axis=1))

submission = pd.DataFrame({'id': df_test_processed['id'], 'y': predictions})
submission.to_csv('submissionLinReg.csv', index=False)